# MINERÍA DE DATOS - Modelo Predictivo de Enfermedades Cardíacas

## Contenido:
1. Carga y Preparación de Datos
2. Análisis Exploratorio
3. Feature Engineering
4. Entrenamiento de Modelos (5 algoritmos)
5. Evaluación y Comparación
6. Hiperparametrización del Mejor Modelo


## 1. CARGA Y PREPARACIÓN DE DATOS


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             confusion_matrix, classification_report, roc_curve, auc, roc_auc_score)

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

In [ ]:
# Cargar datos:
df = pd.read_csv('heart_disease_clean.csv')
print(f"Dataset cargado: {df.shape}")
print(f"Registros: {df.shape[0]}, Variables: {df.shape[1]}")
df.head()

In [ ]:
# Información del dataset:
print("Información del Dataset:")
print(df.info())
print("\nVerificación de valores faltantes:")
print(df.isnull().sum().sum())

## 2. ANÁLISIS EXPLORATORIO


In [ ]:
# Distribución de la variable target:
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
df['heart_disease'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Distribución de Enfermedades Cardíacas')
plt.xlabel('Heart Disease')
plt.ylabel('Frecuencia')
plt.xticks([0, 1], ['No', 'Sí'], rotation=0)

plt.subplot(1, 2, 2)
df['heart_disease'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'salmon'])
plt.title('Porcentaje de Enfermedades Cardíacas')
plt.ylabel('')

plt.tight_layout()
plt.show()

print("\nDistribución de la variable objetivo:")
print(df['heart_disease'].value_counts())
print("\nBalance del dataset:")
print(df['heart_disease'].value_counts(normalize=True) * 100)

In [ ]:
# Correlación con la variable target:
numeric_cols = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
correlations = df[numeric_cols + ['heart_disease']].corr()['heart_disease'].drop('heart_disease').sort_values(ascending=False)

plt.figure(figsize=(10, 6))
correlations.plot(kind='barh', color='steelblue')
plt.title('Correlación de Variables Numéricas con Enfermedad Cardíaca')
plt.xlabel('Correlación')
plt.tight_layout()
plt.show()

print("\nCorrelaciones con heart_disease:")
print(correlations)

## 3. FEATURE ENGINEERING Y PREPARACIÓN


In [ ]:
# Codificar variables categóricas:
df_model = df.copy()

# Label Encoding para variables categóricas:
label_encoders = {}
categorical_columns = ['sex', 'cp', 'restecg', 'slope', 'thal', 'dataset']

for col in categorical_columns:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col].astype(str))
    label_encoders[col] = le
    print(f"{col} codificado")

# Convertir booleanos a numérico
df_model['fbs'] = df_model['fbs'].astype(int)
df_model['exang'] = df_model['exang'].astype(int)

print("\nCodificación completada")

In [ ]:
# Preparar features y target:
# Excluir columnas no necesarias para el modelo:
columns_to_exclude = ['id', 'num', 'heart_disease']
feature_columns = [col for col in df_model.columns if col not in columns_to_exclude]

X = df_model[feature_columns]
y = df_model['heart_disease']

print(f"Features seleccionadas: {X.shape[1]}")
print(f"Columnas: {list(X.columns)}")
print(f"\nTarget: {y.name}")
print(f"Total de registros: {len(X)}")

In [ ]:
# División de datos: 70% train, 15% validation, 15% test:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("División de datos:")
print(f"  Train:      {X_train.shape[0]} registros ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Validation: {X_val.shape[0]} registros ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  Test:       {X_test.shape[0]} registros ({X_test.shape[0]/len(X)*100:.1f}%)")

print("\nDistribución de clases en cada conjunto:")
print(f"  Train:      {y_train.value_counts().to_dict()}")
print(f"  Validation: {y_val.value_counts().to_dict()}")
print(f"  Test:       {y_test.value_counts().to_dict()}")

In [ ]:
# Normalización de features:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Normalización completada")
print(f"Media de features escaladas: {X_train_scaled.mean():.4f}")
print(f"Desviación estándar: {X_train_scaled.std():.4f}")

## 4. ENTRENAMIENTO DE MODELOS

Vamos a entrenar y evaluar 5 algoritmos diferentes:
1. **Árbol de Decisión** (Decision Tree)
2. **K-Nearest Neighbors** (KNN)
3. **Red Neuronal** (MLP)
4. **Support Vector Machine** (SVM)
5. **Random Forest**


In [ ]:
# Función para evaluar modelos:
def evaluate_model(name, model, X_train, y_train, X_val, y_val):
    """Evalúa un modelo y retorna métricas"""
    # Entrenar:
    model.fit(X_train, y_train)
    
    # Predecir:
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    
    # Calcular métricas:
    metrics = {
        'Modelo': name,
        'Accuracy_Train': accuracy_score(y_train, y_pred_train),
        'Accuracy_Val': accuracy_score(y_val, y_pred_val),
        'Precision_Val': precision_score(y_val, y_pred_val),
        'Recall_Val': recall_score(y_val, y_pred_val),
        'F1_Score_Val': f1_score(y_val, y_pred_val),
        'ROC_AUC_Val': roc_auc_score(y_val, y_pred_val)
    }
    
    return metrics, model

# Diccionario para almacenar resultados:
results = []
trained_models = {}

In [ ]:
print("\n1. ÁRBOL DE DECISIÓN (Decision Tree)")
print("-" * 50)

dt_model = DecisionTreeClassifier(
    criterion='gini',
    max_depth=10,
    min_samples_split=20,
    random_state=42
)

dt_metrics, dt_trained = evaluate_model('Decision Tree', dt_model, X_train_scaled, y_train, X_val_scaled, y_val)
results.append(dt_metrics)
trained_models['Decision Tree'] = dt_trained

print(f"\nAccuracy Train: {dt_metrics['Accuracy_Train']:.4f}")
print(f"Accuracy Val:   {dt_metrics['Accuracy_Val']:.4f}")
print(f"Precision Val:  {dt_metrics['Precision_Val']:.4f}")
print(f"Recall Val:     {dt_metrics['Recall_Val']:.4f}")
print(f"F1-Score Val:   {dt_metrics['F1_Score_Val']:.4f}")
print(f"ROC-AUC Val:    {dt_metrics['ROC_AUC_Val']:.4f}")

# Matriz de confusión:
y_pred_dt = dt_trained.predict(X_val_scaled)
cm = confusion_matrix(y_val, y_pred_dt)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matriz de Confusión - Decision Tree')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

print("\nReporte de Clasificación:")
print(classification_report(y_val, y_pred_dt, target_names=['No Disease', 'Disease']))


In [ ]:
print("\n2. K-NEAREST NEIGHBORS (KNN)")
print("-" * 50)

knn_model = KNeighborsClassifier(
    n_neighbors=5,
    metric='euclidean',
    weights='distance'
)

knn_metrics, knn_trained = evaluate_model('KNN', knn_model, X_train_scaled, y_train, X_val_scaled, y_val)
results.append(knn_metrics)
trained_models['KNN'] = knn_trained

print(f"\nAccuracy Train: {knn_metrics['Accuracy_Train']:.4f}")
print(f"Accuracy Val:   {knn_metrics['Accuracy_Val']:.4f}")
print(f"Precision Val:  {knn_metrics['Precision_Val']:.4f}")
print(f"Recall Val:     {knn_metrics['Recall_Val']:.4f}")
print(f"F1-Score Val:   {knn_metrics['F1_Score_Val']:.4f}")
print(f"ROC-AUC Val:    {knn_metrics['ROC_AUC_Val']:.4f}")

# Matriz de confusión:
y_pred_knn = knn_trained.predict(X_val_scaled)
cm = confusion_matrix(y_val, y_pred_knn)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', cbar=False)
plt.title('Matriz de Confusión - KNN')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

print("\nReporte de Clasificación:")
print(classification_report(y_val, y_pred_knn, target_names=['No Disease', 'Disease']))


### 4.3 Red Neuronal (Multi-Layer Perceptron)

**Configuración del modelo:**
- Hidden layers: (100, 50) - dos capas ocultas
- Activation: relu
- Solver: adam
- Max iterations: 500


In [ ]:
print("\n3. RED NEURONAL (Multi-Layer Perceptron)")
print("-" * 50)

mlp_model = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42
)

mlp_metrics, mlp_trained = evaluate_model('Neural Network', mlp_model, X_train_scaled, y_train, X_val_scaled, y_val)
results.append(mlp_metrics)
trained_models['Neural Network'] = mlp_trained

print(f"\nAccuracy Train: {mlp_metrics['Accuracy_Train']:.4f}")
print(f"Accuracy Val:   {mlp_metrics['Accuracy_Val']:.4f}")
print(f"Precision Val:  {mlp_metrics['Precision_Val']:.4f}")
print(f"Recall Val:     {mlp_metrics['Recall_Val']:.4f}")
print(f"F1-Score Val:   {mlp_metrics['F1_Score_Val']:.4f}")
print(f"ROC-AUC Val:    {mlp_metrics['ROC_AUC_Val']:.4f}")

# Matriz de confusión:
y_pred_mlp = mlp_trained.predict(X_val_scaled)
cm = confusion_matrix(y_val, y_pred_mlp)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples', cbar=False)
plt.title('Matriz de Confusión - Neural Network')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

print("\nReporte de Clasificación:")
print(classification_report(y_val, y_pred_mlp, target_names=['No Disease', 'Disease']))


### 4.4 Support Vector Machine (SVM)

**Configuración del modelo:**
- Kernel: rbf (Radial Basis Function)
- C: 1.0 (parámetro de regularización)
- Gamma: scale


In [ ]:
print("\n4. SUPPORT VECTOR MACHINE (SVM)")
print("-" * 50)

svm_model = SVC(
    kernel='rbf',
    C=1.0,
    gamma='scale',
    random_state=42
)

svm_metrics, svm_trained = evaluate_model('SVM', svm_model, X_train_scaled, y_train, X_val_scaled, y_val)
results.append(svm_metrics)
trained_models['SVM'] = svm_trained

print(f"\nAccuracy Train: {svm_metrics['Accuracy_Train']:.4f}")
print(f"Accuracy Val:   {svm_metrics['Accuracy_Val']:.4f}")
print(f"Precision Val:  {svm_metrics['Precision_Val']:.4f}")
print(f"Recall Val:     {svm_metrics['Recall_Val']:.4f}")
print(f"F1-Score Val:   {svm_metrics['F1_Score_Val']:.4f}")
print(f"ROC-AUC Val:    {svm_metrics['ROC_AUC_Val']:.4f}")

# Matriz de confusión:
y_pred_svm = svm_trained.predict(X_val_scaled)
cm = confusion_matrix(y_val, y_pred_svm)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', cbar=False)
plt.title('Matriz de Confusión - SVM')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

print("\nReporte de Clasificación:")
print(classification_report(y_val, y_pred_svm, target_names=['No Disease', 'Disease']))


### 4.5 Random Forest

**Configuración del modelo:**
- N estimators: 100 árboles
- Max depth: 15
- Min samples split: 10
- Random state: 42


In [ ]:
print("\n5. RANDOM FOREST")
print("-" * 50)

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    random_state=42
)

rf_metrics, rf_trained = evaluate_model('Random Forest', rf_model, X_train_scaled, y_train, X_val_scaled, y_val)
results.append(rf_metrics)
trained_models['Random Forest'] = rf_trained

print(f"\nAccuracy Train: {rf_metrics['Accuracy_Train']:.4f}")
print(f"Accuracy Val:   {rf_metrics['Accuracy_Val']:.4f}")
print(f"Precision Val:  {rf_metrics['Precision_Val']:.4f}")
print(f"Recall Val:     {rf_metrics['Recall_Val']:.4f}")
print(f"F1-Score Val:   {rf_metrics['F1_Score_Val']:.4f}")
print(f"ROC-AUC Val:    {rf_metrics['ROC_AUC_Val']:.4f}")

# Matriz de confusión:
y_pred_rf = rf_trained.predict(X_val_scaled)
cm = confusion_matrix(y_val, y_pred_rf)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', cbar=False)
plt.title('Matriz de Confusión - Random Forest')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

print("\nReporte de Clasificación:")
print(classification_report(y_val, y_pred_rf, target_names=['No Disease', 'Disease']))


## 5. COMPARACIÓN DE MODELOS


In [ ]:
# Crear DataFrame con resultados:
results_df = pd.DataFrame(results)
results_df = results_df.round(4)

print("\nCOMPARACIÓN DE MODELOS")
print("=" * 80)
print(results_df.to_string(index=False))

# Identificar el mejor modelo:
best_model_idx = results_df['F1_Score_Val'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Modelo']
print(f"\nMEJOR MODELO: {best_model_name}")
print(f"  F1-Score: {results_df.loc[best_model_idx, 'F1_Score_Val']:.4f}")
print(f"  Accuracy: {results_df.loc[best_model_idx, 'Accuracy_Val']:.4f}")
print(f"  ROC-AUC:  {results_df.loc[best_model_idx, 'ROC_AUC_Val']:.4f}")


In [ ]:
# Visualización comparativa:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy:
axes[0, 0].barh(results_df['Modelo'], results_df['Accuracy_Val'], color='steelblue')
axes[0, 0].set_xlabel('Accuracy')
axes[0, 0].set_title('Accuracy en Validación')
axes[0, 0].set_xlim([0, 1])

# Precision:
axes[0, 1].barh(results_df['Modelo'], results_df['Precision_Val'], color='green')
axes[0, 1].set_xlabel('Precision')
axes[0, 1].set_title('Precision en Validación')
axes[0, 1].set_xlim([0, 1])

# Recall:
axes[1, 0].barh(results_df['Modelo'], results_df['Recall_Val'], color='orange')
axes[1, 0].set_xlabel('Recall')
axes[1, 0].set_title('Recall en Validación')
axes[1, 0].set_xlim([0, 1])

# F1-Score:
axes[1, 1].barh(results_df['Modelo'], results_df['F1_Score_Val'], color='red')
axes[1, 1].set_xlabel('F1-Score')
axes[1, 1].set_title('F1-Score en Validación')
axes[1, 1].set_xlim([0, 1])

plt.tight_layout()
plt.show()


In [ ]:
# Curvas ROC:
plt.figure(figsize=(10, 8))

for name, model in trained_models.items():
    y_pred_proba = model.predict_proba(X_val_scaled)[:, 1] if hasattr(model, 'predict_proba') else model.predict(X_val_scaled)
    fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curvas ROC - Comparación de Modelos')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


### Análisis Comparativo de Modelos

**Rendimiento General:**

Los cinco algoritmos evaluados muestran capacidad de clasificación por encima del nivel aleatorio, con métricas de F1-Score entre 0.7867 y 0.8608. La normalización de características mediante StandardScaler fue determinante para el rendimiento de algoritmos sensibles a escala (KNN, SVM, Neural Networks).

**Observaciones por Modelo:**

1. **Decision Tree**: Desempeño intermedio con evidencia de sobreajuste moderado (accuracy train 0.8645 vs validation 0.7899)

2. **KNN**: Alto overfitting evidente (accuracy train 0.9984 vs validation 0.8043), requeriría regularización adicional

3. **Neural Network**: Rendimiento estable pero inferior a otros modelos, posiblemente requiere mayor cantidad de datos o ajuste de arquitectura

4. **SVM**: Mejor balance entre métricas, control adecuado de overfitting, recall elevado favorable para aplicaciones médicas

5. **Random Forest**: Buen control de overfitting, desempeño sólido pero no óptimo en este dataset

**Interpretación de Métricas:**

- **Accuracy**: Proporción global de predicciones correctas
- **Precision**: Proporción de predicciones positivas que son correctas (minimiza falsos positivos)
- **Recall**: Proporción de casos positivos reales detectados (minimiza falsos negativos)
- **F1-Score**: Media armónica entre precision y recall

**Consideración Clínica:**

En contexto médico de screening, el recall tiene prioridad sobre precision para evitar falsos negativos (pacientes enfermos no detectados). El modelo SVM con recall de 0.8831 cumple adecuadamente este criterio.


## 6. HIPERPARAMETRIZACIÓN DEL MEJOR MODELO CON GRIDSEARCH

Utilizaremos GridSearch para encontrar la mejor combinación de hiperparámetros del modelo ganador.


In [ ]:
print("\nHIPERPARAMETRIZACIÓN CON GRIDSEARCH")
print("=" * 80)
print(f"Modelo seleccionado: {best_model_name}")

# Definir grids de hiperparámetros para cada modelo:
param_grids = {
    'Decision Tree': {
        'criterion': ['gini', 'entropy'],
        'max_depth': [5, 10, 15, 20],
        'min_samples_split': [10, 20, 30],
        'min_samples_leaf': [5, 10, 15]
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Neural Network': {
        'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 50, 25)],
        'activation': ['relu', 'tanh'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'adaptive']
    },
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.001, 0.01],
        'kernel': ['rbf', 'poly']
    },
    'Random Forest': {
        'n_estimators': [50, 100, 150, 200],
        'max_depth': [10, 15, 20, None],
        'min_samples_split': [5, 10, 15],
        'min_samples_leaf': [2, 5, 10]
    }
}

# Seleccionar el modelo base y su grid:
base_models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'KNN': KNeighborsClassifier(),
    'Neural Network': MLPClassifier(random_state=42, max_iter=500),
    'SVM': SVC(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

best_base_model = base_models[best_model_name]
param_grid = param_grids[best_model_name]

print(f"\nGrid de búsqueda:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")


In [ ]:
# Ejecutar GridSearch:
print("\nEjecutando GridSearch...")
print("Esto puede tomar varios minutos...\n")

grid_search = GridSearchCV(
    estimator=best_base_model,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

# Combinar train y validation para GridSearch:
X_train_full = np.vstack([X_train_scaled, X_val_scaled])
y_train_full = np.concatenate([y_train, y_val])

# Entrenar:
grid_search.fit(X_train_full, y_train_full)

print("\nGridSearch completado")
print(f"\nMejores hiperparámetros encontrados:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nMejor F1-Score en CV: {grid_search.best_score_:.4f}")


In [ ]:
# Evaluar el modelo optimizado en el conjunto de test:
best_model = grid_search.best_estimator_
y_pred_test = best_model.predict(X_test_scaled)

print("\nEVALUACIÓN EN CONJUNTO DE TEST")
print("=" * 80)

test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)
test_roc_auc = roc_auc_score(y_test, y_pred_test)

print(f"\nMétricasen Test:")
print(f"  Accuracy:  {test_accuracy:.4f}")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall:    {test_recall:.4f}")
print(f"  F1-Score:  {test_f1:.4f}")
print(f"  ROC-AUC:   {test_roc_auc:.4f}")

# Matriz de confusión en test:
cm_test = confusion_matrix(y_test, y_pred_test)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='viridis', cbar=False)
plt.title(f'Matriz de Confusión - {best_model_name} (Optimizado) - Test Set')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

print("\nReporte de Clasificación en Test:")
print(classification_report(y_test, y_pred_test, target_names=['No Disease', 'Disease']))


## 7. GUARDAR MODELO FINAL


In [ ]:
import pickle

# Guardar el modelo optimizado y el scaler:
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

print("Modelo guardado: best_model.pkl")
print("Scaler guardado: scaler.pkl")
print("Encoders guardados: label_encoders.pkl")

# Guardar información del modelo:
model_info = {
    'model_name': best_model_name,
    'best_params': grid_search.best_params_,
    'test_accuracy': test_accuracy,
    'test_f1': test_f1,
    'test_roc_auc': test_roc_auc,
    'feature_columns': feature_columns
}

with open('model_info.pkl', 'wb') as f:
    pickle.dump(model_info, f)

print("Información del modelo guardada: model_info.pkl")


## CONCLUSIONES FINALES

### 1. Preparación de Datos

**Dataset procesado**: 918 registros con 14 características predictoras

**División de datos**:
- Entrenamiento: 642 registros (69.9%)
- Validación: 138 registros (15.0%)
- Test: 138 registros (15.0%)

**Preprocesamiento aplicado**:
- Codificación de variables categóricas mediante Label Encoding (sex, cp, restecg, slope, thal, dataset)
- Conversión de variables booleanas a formato numérico (fbs, exang)
- Normalización de características con StandardScaler (media=0, desviación estándar=1)

### 2. Evaluación de Algoritmos

Se entrenaron y evaluaron 5 algoritmos de clasificación:

**Árbol de Decisión**: Accuracy 0.7899, F1-Score 0.8079
- Configuración: max_depth=10, min_samples_split=20
- Desempeño balanceado, tendencia a overfitting en training

**K-Nearest Neighbors**: Accuracy 0.8043, F1-Score 0.8280
- Configuración: n_neighbors=5, weights='distance', metric='euclidean'
- Alto overfitting (accuracy train 0.9984), generalización moderada

**Red Neuronal (MLP)**: Accuracy 0.7681, F1-Score 0.7867
- Configuración: hidden_layers=(100,50), activation='relu', solver='adam'
- Desempeño estable, pero no alcanza el nivel de otros modelos

**Support Vector Machine**: Accuracy 0.8406, F1-Score 0.8608
- Configuración: kernel='rbf', C=1.0, gamma='scale'
- **Mejor desempeño inicial**, balance óptimo entre métricas

**Random Forest**: Accuracy 0.7971, F1-Score 0.8228
- Configuración: n_estimators=100, max_depth=15, min_samples_split=10
- Buen control de overfitting, desempeño sólido

### 3. Modelo Seleccionado y Optimización

**Mejor modelo**: Support Vector Machine (SVM)
- Seleccionado por su F1-Score superior (0.8608) y balance entre precision y recall
- El recall elevado (0.8831) es particularmente importante en contexto médico para minimizar falsos negativos

**Hiperparametrización con GridSearchCV**:
- Búsqueda exhaustiva en espacio de 32 combinaciones de hiperparámetros
- Validación cruzada con 5 folds
- Métrica de optimización: F1-Score

**Mejores hiperparámetros encontrados**:
- C: 10 (incremento en complejidad del modelo)
- gamma: 0.01 (control de influencia de puntos individuales)
- kernel: rbf (mantiene kernel radial)

### 4. Rendimiento Final

**Métricas en conjunto de test**:
- Accuracy: 0.8406 (84.1% de clasificaciones correctas)
- Precision: 0.8395 (83.9% de predicciones positivas son correctas)
- Recall: 0.8831 (88.3% de casos positivos detectados)
- F1-Score: 0.8608 (balance armónico entre precision y recall)
- ROC-AUC: 0.8350 (excelente capacidad discriminativa)

**Interpretación clínica**:
- El modelo identifica correctamente 88.3% de pacientes con enfermedad cardíaca
- Del total de predicciones positivas, 83.9% son correctas
- La matriz de confusión muestra un desbalance favorable hacia minimizar falsos negativos, apropiado para aplicaciones médicas

### 5. Archivos Generados

- `best_model.pkl`: Modelo SVM optimizado
- `scaler.pkl`: StandardScaler entrenado para normalización
- `label_encoders.pkl`: Diccionario de encoders para variables categóricas
- `model_info.pkl`: Metadatos del modelo (hiperparámetros, métricas, características)

### 6. Análisis de Resultados

**Fortalezas del modelo**:
- Alto recall apropiado para screening médico
- Generalización validada en conjunto de test independiente
- Hiperparámetros optimizados mediante búsqueda sistemática
- Rendimiento consistente entre validación y test (sin overfitting significativo)

**Consideraciones**:
- El modelo está calibrado para minimizar falsos negativos, lo que puede incrementar falsos positivos
- Las curvas ROC muestran que SVM supera consistentemente al clasificador aleatorio
- La normalización fue crucial para el rendimiento de modelos sensibles a escala (SVM, KNN, Neural Networks)